In [1]:
# -*- coding: utf-8 -*-
# Complete, runnable TabTransformer pipeline with automatic preprocessing for string categorical columns.
# Single-class TabTransformer (Backbone + Head merged)

# Third Party
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss
from torch.utils.data import DataLoader, TensorDataset

# ---------------------------------------------------------
# 0) Preprocessor: string categorical -> integer IDs (with OOV=0)
#    + auto cat_idx/cont_idx + cardinalities
# ---------------------------------------------------------
class TabularPreprocessor:
    def __init__(self, categorical_indices=None, use_oov=True, oov_token=0, add_na_token=True):
        self.categorical_indices = None if categorical_indices is None else list(categorical_indices)
        self.use_oov = use_oov
        self.oov_token = int(oov_token)  # 0 recommended
        self.add_na_token = add_na_token
        self.cat_maps = {}          # col_idx -> {category_value: int_id}
        self.cardinalities = []     # per-categorical-column unique count (K, excluding OOV)
        self.cat_idx = []
        self.cont_idx = []
        self.fitted_ = False

    def _ensure_ndarray(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.values
        return np.asarray(X, dtype=object)  # safe for mixed types

    def fit(self, X, categorical_indices=None):
        X = self._ensure_ndarray(X)
        n_cols = X.shape[1]

        if categorical_indices is not None:
            self.categorical_indices = list(categorical_indices)

        if self.categorical_indices is None:
            # fallback: infer by dtype==object
            self.categorical_indices = [j for j in range(n_cols) if X[:, j].dtype == object]

        cat_set = set(self.categorical_indices)
        self.cat_idx = sorted(list(cat_set))
        self.cont_idx = [j for j in range(n_cols) if j not in cat_set]

        # build per-column maps: real categories 1..K (0 reserved for OOV)
        self.cat_maps = {}
        self.cardinalities = []
        for j in self.cat_idx:
            col = X[:, j]
            if self.add_na_token:
                col = np.where(pd.isna(col), "<NA>", col)
            uniques = pd.unique(col)
            id_map = {val: i + 1 for i, val in enumerate(uniques)}  # 1..K
            self.cat_maps[j] = id_map
            self.cardinalities.append(len(uniques))  # exclude OOV

        self.fitted_ = True
        return self

    def transform(self, X):
        assert self.fitted_, "Call fit() before transform()."
        X = self._ensure_ndarray(X)

        # categorical
        if len(self.cat_idx) > 0:
            x_cat = np.zeros((X.shape[0], len(self.cat_idx)), dtype="int64")
            for ti, j in enumerate(self.cat_idx):
                col = X[:, j]
                if self.add_na_token:
                    col = np.where(pd.isna(col), "<NA>", col)
                id_map = self.cat_maps[j]
                x_cat[:, ti] = np.array([id_map.get(v, self.oov_token) for v in col], dtype="int64")
        else:
            x_cat = np.zeros((X.shape[0], 0), dtype="int64")

        # continuous
        if len(self.cont_idx) > 0:
            x_cont = X[:, self.cont_idx].astype("float32")
        else:
            x_cont = None

        return x_cat, x_cont

    def fit_transform(self, X, categorical_indices=None):
        self.fit(X, categorical_indices)
        return self.transform(X)


# ---------------------------------------------------------
# 1) Single-class TabTransformer (Backbone + Continuous + Head)
# ---------------------------------------------------------
class TabTransformerNet(nn.Module):
    """
    Single-class TabTransformer:
    - Categorical: per-column embedding (+ OOV/pad), column embedding, Transformer encoder, pooling
    - Continuous: BatchNorm + (optional) Linear projection to d_token
    - Head: MLP -> sigmoid
    """
    def __init__(
        self,
        cat_cardinalities,          # List[int] (exclude OOV)
        n_continuous=0,
        d_token=32,
        n_heads=4,
        n_layers=2,
        dim_feedforward=128,
        attn_dropout=0.1,
        embedding_dropout=0.1,
        add_cls=False,
        pooling="concat",           # "concat" or "cls"
        cont_proj="linear",         # "none" or "linear"
        mlp_hidden_dims=(128, 64),
        mlp_dropout=0.2,
        padding_idx=0,              # reserve 0 for OOV/pad if not None
        norm_first=True
    ):
        super().__init__()
        assert pooling in ("concat", "cls")
        self.n_cat = len(cat_cardinalities)
        self.n_cont = n_continuous
        self.d_token = d_token
        self.add_cls = add_cls
        self.pooling = pooling
        self.cont_proj = cont_proj

        # ---- Categorical path ----
        if self.n_cat == 0:
            self.cat_embeddings = nn.ModuleList()
            self.col_embedding = None
        else:
            # +1 slot if padding_idx is used (OOV/pad)
            self.cat_embeddings = nn.ModuleList([
                nn.Embedding(
                    num_embeddings=c + (1 if (padding_idx is not None) else 0),
                    embedding_dim=d_token,
                    padding_idx=padding_idx
                )
                for c in cat_cardinalities
            ])
            self.col_embedding = nn.Embedding(self.n_cat, d_token)

        if self.add_cls:
            self.cls_token = nn.Parameter(torch.zeros(1, 1, d_token))
            nn.init.normal_(self.cls_token, std=0.02)

        self.embedding_dropout = nn.Dropout(embedding_dropout)

        enc_layer = nn.TransformerEncoderLayer(
            d_model=d_token,
            nhead=n_heads,
            dim_feedforward=dim_feedforward,
            dropout=attn_dropout,
            batch_first=True,
            norm_first=norm_first
        )
        self.transformer = nn.TransformerEncoder(enc_layer, num_layers=n_layers)

        # init categorical embeddings
        for emb in self.cat_embeddings:
            nn.init.normal_(emb.weight, std=0.02)
        if self.col_embedding is not None:
            nn.init.normal_(self.col_embedding.weight, std=0.02)

        # ---- Continuous path ----
        if n_continuous > 0:
            self.cont_bn = nn.BatchNorm1d(n_continuous)
            if cont_proj == "linear":
                self.cont_linear = nn.Linear(n_continuous, d_token)
                nn.init.kaiming_uniform_(self.cont_linear.weight, mode="fan_in", nonlinearity="relu")
                cont_out_dim = d_token
            else:
                self.cont_linear = nn.Identity()
                cont_out_dim = n_continuous
        else:
            self.cont_bn = None
            self.cont_linear = None
            cont_out_dim = 0

        # ---- Head ----
        backbone_out = (d_token if pooling == "cls" else self.n_cat * d_token)
        in_dim = backbone_out + cont_out_dim

        layers = []
        prev = in_dim
        for h in mlp_hidden_dims:
            lin = nn.Linear(prev, h)
            nn.init.kaiming_uniform_(lin.weight, mode="fan_in", nonlinearity="relu")
            layers.extend([lin, nn.BatchNorm1d(h), nn.ReLU(), nn.Dropout(mlp_dropout)])
            prev = h
        layers.append(nn.Linear(prev, 1))
        layers.append(nn.Sigmoid())
        self.head = nn.Sequential(*layers)

    def _encode_categoricals(self, x_cat: torch.LongTensor):
        """
        x_cat: (B, n_cat) -> contextualized representation
        returns:
          pooling='concat' -> (B, n_cat*d)
          pooling='cls'    -> (B, d)
        """
        B = x_cat.size(0)
        if self.n_cat == 0:
            return torch.zeros(B, self.d_token if self.pooling == "cls" else 0,
                               device=x_cat.device, dtype=torch.float32)

        tok_list = []
        for j, emb in enumerate(self.cat_embeddings):
            tok = emb(x_cat[:, j])                         # (B, d)
            if self.col_embedding is not None:
                tok = tok + self.col_embedding.weight[j]   # (d,)
            tok_list.append(tok.unsqueeze(1))              # (B, 1, d)

        x_tok = torch.cat(tok_list, dim=1)                 # (B, n_cat, d)

        if self.add_cls:
            cls = self.cls_token.expand(B, -1, -1)         # (B, 1, d)
            x_tok = torch.cat([cls, x_tok], dim=1)         # (B, 1+n_cat, d)

        x_tok = self.embedding_dropout(x_tok)
        z = self.transformer(x_tok)                        # (B, T, d)

        if self.pooling == "cls" and self.add_cls:
            out = z[:, 0, :]                               # (B, d)
        elif self.pooling == "cls":
            out = z.mean(dim=1)                            # (B, d)
        else:
            if self.add_cls:
                z = z[:, 1:, :]                            # drop CLS
            out = z.reshape(B, -1)                         # (B, n_cat*d)
        return out

    def forward(self, x_cat: torch.LongTensor, x_cont: torch.FloatTensor = None):
        z_cat = self._encode_categoricals(x_cat)
        if (x_cont is not None) and (self.n_cont > 0):
            if x_cont.ndim == 1:
                x_cont = x_cont.unsqueeze(1)
            x_cont = self.cont_bn(x_cont)
            x_cont = self.cont_linear(x_cont)
            z = torch.cat([z_cat, x_cont], dim=1)
        else:
            z = z_cat
        out = self.head(z)
        return out


# ---------------------------------------------------------
# 2) Sklearn-Compatible Classifier with Auto Preprocessing
# ---------------------------------------------------------
class TabTransformerBinaryClassifier(BaseEstimator, ClassifierMixin):
    """
    Example:
        clf = TabTransformerBinaryClassifier(
            auto_preprocess=True,
            categorical_indices=[0, 3, 5],
            d_token=32, n_heads=4, n_layers=2, ...
        )
    """
    def __init__(
        self,
        cat_idx=None,
        cat_cardinalities=None,
        cont_idx=None,
        d_token=32,
        n_heads=4,
        n_layers=2,
        dim_feedforward=128,
        attn_dropout=0.1,
        embedding_dropout=0.1,
        add_cls=False,
        pooling="concat",
        cont_proj="linear",
        hidden_dims=(128, 64),
        mlp_dropout=0.2,
        lr=1e-3,
        weight_decay=1e-4,
        loss_fn="logloss",
        device=None,
        # Auto preprocessing
        auto_preprocess=True,
        categorical_indices=None,
        use_oov=True
    ):
        # manual fields (kept for compatibility)
        self.cat_idx = [] if cat_idx is None else list(cat_idx)
        self.cat_cardinalities = None if cat_cardinalities is None else list(cat_cardinalities)
        self.cont_idx = [] if cont_idx is None else list(cont_idx)

        # model/training params
        self.d_token = d_token
        self.n_heads = n_heads
        self.n_layers = n_layers
        self.dim_feedforward = dim_feedforward
        self.attn_dropout = attn_dropout
        self.embedding_dropout = embedding_dropout
        self.add_cls = add_cls
        self.pooling = pooling
        self.cont_proj = cont_proj
        self.hidden_dims = hidden_dims
        self.mlp_dropout = mlp_dropout
        self.lr = lr
        self.weight_decay = weight_decay
        self.loss_fn_name = loss_fn

        self.model = None
        self.best_model_weights = None
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # auto-preproc
        self.auto_preprocess = auto_preprocess
        self.categorical_indices = categorical_indices
        self.use_oov = use_oov
        self.preproc = None

    def _build_model(self):
        n_cont = len(self.cont_idx)
        padding_idx = 0 if self.use_oov else None
        model = TabTransformerNet(
            cat_cardinalities=self.cat_cardinalities,
            n_continuous=n_cont,
            d_token=self.d_token,
            n_heads=self.n_heads,
            n_layers=self.n_layers,
            dim_feedforward=self.dim_feedforward,
            attn_dropout=self.attn_dropout,
            embedding_dropout=self.embedding_dropout,
            add_cls=self.add_cls,
            pooling=self.pooling,
            cont_proj=self.cont_proj,
            mlp_hidden_dims=self.hidden_dims,
            mlp_dropout=self.mlp_dropout,
            padding_idx=padding_idx,
            norm_first=True
        )
        return model.to(self.device)

    def _define_loss_fn(self):
        if self.loss_fn_name == "logloss":
            return nn.BCELoss(reduction="none")
        else:
            raise Exception(f"{self.loss_fn_name} is not defined")

    def _split_X(self, X):
        """
        If auto_preprocess=True -> use preproc.transform(X).
        Else -> use provided indices.
        """
        if self.auto_preprocess:
            assert self.preproc is not None and self.preproc.fitted_, "Call fit() first."
            x_cat_np, x_cont_np = self.preproc.transform(X)
        else:
            X_np = X.detach().cpu().numpy() if isinstance(X, torch.Tensor) else X
            if len(self.cat_idx) > 0:
                x_cat_np = X_np[:, self.cat_idx].astype("int64")
            else:
                x_cat_np = np.zeros((X_np.shape[0], 0), dtype="int64")
            x_cont_np = (X_np[:, self.cont_idx].astype("float32") if len(self.cont_idx) > 0 else None)

        x_cat = torch.tensor(x_cat_np, dtype=torch.long, device=self.device)
        x_cont = torch.tensor(x_cont_np, dtype=torch.float32, device=self.device) if x_cont_np is not None else None
        return x_cat, x_cont

    def fit(
        self,
        X,
        y,
        sample_weight=None,
        eval_set=None,            # list of tuples: [(X_val, y_val)]
        eval_metric=None,         # supports ["logloss"]
        max_epochs=10,
        patience=None,
        batch_size=32,
        num_workers=0,
        verbose=True,
        pin_memory=None
    ):
        if pin_memory is None:
            pin_memory = (self.device == "cuda")

        # Auto-preprocessing: infer indices + cardinalities from data
        if self.auto_preprocess:
            self.preproc = TabularPreprocessor(
                categorical_indices=self.categorical_indices,
                use_oov=self.use_oov,
                oov_token=0,
                add_na_token=True
            )
            self.preproc.fit(X)  # derive cat_idx/cont_idx/cardinalities
            self.cat_idx = self.preproc.cat_idx
            self.cont_idx = self.preproc.cont_idx
            self.cat_cardinalities = self.preproc.cardinalities

        # Tensors
        x_cat, x_cont = self._split_X(X)
        y_tensor = torch.tensor(y, dtype=torch.float32, device=self.device).view(-1, 1)
        w_tensor = (torch.tensor(sample_weight, dtype=torch.float32, device=self.device).view(-1, 1)
                    if sample_weight is not None else torch.ones_like(y_tensor))

        # Validation
        if eval_set is not None:
            X_val, y_val = eval_set[0]
            x_cat_val, x_cont_val = self._split_X(X_val)
            y_val_tensor = torch.tensor(y_val, dtype=torch.float32, device=self.device).view(-1, 1)
        else:
            x_cat_val = x_cont_val = y_val_tensor = None

        # Build model
        if self.model is None:
            self.model = self._build_model()

        loss_fn = self._define_loss_fn()
        optimizer = optim.Adam(self.model.parameters(), lr=self.lr, weight_decay=self.weight_decay)

        # DataLoaders
        if x_cont is None:
            train_dataset = TensorDataset(x_cat, y_tensor, w_tensor)
            def _iter(loader):
                for Xc_b, y_b, w_b in loader:
                    yield Xc_b, None, y_b, w_b
        else:
            train_dataset = TensorDataset(x_cat, x_cont, y_tensor, w_tensor)
            def _iter(loader):
                for Xc_b, Xn_b, y_b, w_b in loader:
                    yield Xc_b, Xn_b, y_b, w_b

        train_loader = DataLoader(
            train_dataset, batch_size=batch_size, shuffle=True,
            num_workers=num_workers, pin_memory=pin_memory
        )

        best_loss, patience_counter = float("inf"), 0

        for epoch in range(max_epochs):
            self.model.train()
            epoch_loss, n_steps = 0.0, 0

            for Xc_b, Xn_b, y_b, w_b in _iter(train_loader):
                optimizer.zero_grad()
                y_pred = self.model(Xc_b, Xn_b)
                loss = loss_fn(y_pred, y_b)
                (loss * w_b).sum().div(w_b.sum()).backward()
                optimizer.step()

                epoch_loss += (loss * w_b).sum().div(w_b.sum()).item()
                n_steps += 1

            if verbose:
                print(f"Epoch {epoch + 1}/{max_epochs} - train_loss: {epoch_loss / max(1, n_steps):.6f}")

            # ---- Validation ----
            if eval_set is not None:
                self.model.eval()
                with torch.no_grad():
                    if x_cont_val is None:
                        val_ds = TensorDataset(x_cat_val, y_val_tensor)
                        val_loader = DataLoader(val_ds, batch_size=2048, shuffle=False,
                                                num_workers=num_workers, pin_memory=pin_memory)
                        eval_loss, n_eval = 0.0, 0
                        for Xc_v, y_v in val_loader:
                            y_p = self.model(Xc_v, None)
                            lv = loss_fn(y_p, y_v)
                            eval_loss += (lv.sum() / len(lv)).item()
                            n_eval += 1
                    else:
                        val_ds = TensorDataset(x_cat_val, x_cont_val, y_val_tensor)
                        val_loader = DataLoader(val_ds, batch_size=2048, shuffle=False,
                                                num_workers=num_workers, pin_memory=pin_memory)
                        eval_loss, n_eval = 0.0, 0
                        for Xc_v, Xn_v, y_v in val_loader:
                            y_p = self.model(Xc_v, Xn_v)
                            lv = loss_fn(y_p, y_v)
                            eval_loss += (lv.sum() / len(lv)).item()
                            n_eval += 1
                    eval_loss /= max(1, n_eval)
                    if verbose:
                        print(f"          val_loss: {eval_loss:.6f}")

                    if patience is not None:
                        if eval_loss < best_loss:
                            best_loss = eval_loss
                            patience_counter = 0
                            self.best_model_weights = {k: v.detach().cpu().clone()
                                                       for k, v in self.model.state_dict().items()}
                        else:
                            patience_counter += 1
                            if patience_counter >= patience:
                                if verbose:
                                    print(f"Early stopping at epoch {epoch + 1}")
                                break

        if self.best_model_weights is not None:
            self.model.load_state_dict(self.best_model_weights)

        return self

    def predict_proba(self, X):
        x_cat, x_cont = self._split_X(X)
        self.model.eval()
        with torch.no_grad():
            probs1 = self.model(x_cat, x_cont).detach().cpu().numpy()  # (B, 1)
        probs1 = probs1.astype("float")
        probs0 = 1.0 - probs1
        return np.hstack([probs0, probs1])

    def predict(self, X):
        probs = self.predict_proba(X)
        return probs.argmax(axis=1)


# ---------------------------------------------------------
# 3) Sample mixed-type dataset generator (with strings)
# ---------------------------------------------------------
def make_mixed_sample(
    n_samples=40000,
    seed=7
):
    rng = np.random.RandomState(seed)

    genders = np.array(["남성", "여성", "기타"], dtype=object)
    cities = np.array(["서울", "부산", "대구", "인천", "수원", "고양"], dtype=object)
    devices = np.array(["ios", "android", "web"], dtype=object)

    # categorical columns
    gender_col = rng.choice(genders, size=n_samples, p=[0.48, 0.48, 0.04])
    city_col   = rng.choice(cities, size=n_samples)
    device_col = rng.choice(devices, size=n_samples, p=[0.35, 0.55, 0.10])

    # continuous columns
    n_cont = 10
    X_cont = rng.randn(n_samples, n_cont).astype("float32")

    # latent score
    w_gender = {g: w for g, w in zip(genders, rng.uniform(-0.8, 0.8, size=len(genders)))}
    w_city   = {c: w for c, w in zip(cities, rng.uniform(-0.6, 1.0, size=len(cities)))}
    w_device = {d: w for d, w in zip(devices, rng.uniform(-0.5, 0.9, size=len(devices)))}
    w_cont   = rng.randn(n_cont).astype("float32")

    score_cat = (np.vectorize(lambda v: w_gender[v])(gender_col) +
                 np.vectorize(lambda v: w_city[v])(city_col) +
                 np.vectorize(lambda v: w_device[v])(device_col)).astype("float32")
    score_cont = (X_cont * w_cont).sum(axis=1).astype("float32")

    bias = 0.1
    noise = rng.normal(scale=0.5, size=n_samples).astype("float32")
    logit = 0.7 * score_cat + 0.8 * score_cont + bias + noise
    prob = 1.0 / (1.0 + np.exp(-logit))
    y = (prob > 0.5).astype("int64")

    # combine into feature array (object dtype)
    X = np.empty((n_samples, 3 + n_cont), dtype=object)
    X[:, 0] = gender_col
    X[:, 1] = city_col
    X[:, 2] = device_col
    X[:, 3:] = X_cont

    categorical_feature_indices = [0, 1, 2]
    return X, y, categorical_feature_indices


# ---------------------------------------------------------
# 4) Train & Evaluate Demo
# ---------------------------------------------------------
def train_and_evaluate_demo():
    np.random.seed(0)
    torch.manual_seed(0)

    # ----- Data -----
    X, y, categorical_feature_indices = make_mixed_sample(n_samples=30000, seed=123)

    # train/val/test split
    N = X.shape[0]
    idx = np.arange(N)
    np.random.shuffle(idx)

    tr_end = int(N * 0.7)
    va_end = int(N * 0.85)
    tr_idx, va_idx, te_idx = idx[:tr_end], idx[tr_end:va_end], idx[va_end:]

    X_tr, y_tr = X[tr_idx], y[tr_idx]
    X_va, y_va = X[va_idx], y[va_idx]
    X_te, y_te = X[te_idx], y[te_idx]

    # ----- Model -----
    clf = TabTransformerBinaryClassifier(
        auto_preprocess=True,
        categorical_indices=categorical_feature_indices,
        use_oov=True,                # reserve 0 for OOV/pad
        # model hyperparams
        d_token=32,
        n_heads=4,
        n_layers=2,
        dim_feedforward=128,
        attn_dropout=0.1,
        embedding_dropout=0.05,
        pooling="concat",
        add_cls=False,
        cont_proj="linear",
        hidden_dims=(128, 64),
        mlp_dropout=0.2,
        lr=1e-3,
        weight_decay=1e-4,
        loss_fn="logloss",
        device="cuda" if torch.cuda.is_available() else "cpu"
    )

    clf.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric=["logloss"],
        max_epochs=10,
        patience=2,
        batch_size=1024,     # adjust per GPU/CPU memory
        num_workers=0,
        verbose=True
    )

    # ----- Eval -----
    proba_te = clf.predict_proba(X_te)[:, 1]
    pred_te = (proba_te >= 0.5).astype(int)

    acc = accuracy_score(y_te, pred_te)
    auc = roc_auc_score(y_te, proba_te)
    ll  = log_loss(y_te, np.vstack([1 - proba_te, proba_te]).T)

    print("\n===== Test Metrics =====")
    print(f"Accuracy : {acc:.4f}")
    print(f"ROC-AUC  : {auc:.4f}")
    print(f"Logloss  : {ll:.4f}")

    print("\nSample predictions (first 10):")
    print(np.round(proba_te[:10], 4))

    if clf.auto_preprocess:
        print("\n[Info] Derived categorical indices:", clf.cat_idx)
        print("[Info] Derived continuous indices :", clf.cont_idx[:10], "... (total:", len(clf.cont_idx), ")")
        print("[Info] Cardinalities (per cat col):", clf.cat_cardinalities)

if __name__ == "__main__":
    train_and_evaluate_demo()

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


Epoch 1/10 - train_loss: 0.552070
          val_loss: 0.384910
Epoch 2/10 - train_loss: 0.333158
          val_loss: 0.254635
Epoch 3/10 - train_loss: 0.231352
          val_loss: 0.198390
Epoch 4/10 - train_loss: 0.184456
          val_loss: 0.162576
Epoch 5/10 - train_loss: 0.156747
          val_loss: 0.154019
Epoch 6/10 - train_loss: 0.143633
          val_loss: 0.139364
Epoch 7/10 - train_loss: 0.135302
          val_loss: 0.143477
Epoch 8/10 - train_loss: 0.130429
          val_loss: 0.137863
Epoch 9/10 - train_loss: 0.128534
          val_loss: 0.129339
Epoch 10/10 - train_loss: 0.124611
          val_loss: 0.130620

===== Test Metrics =====
Accuracy : 0.9436
ROC-AUC  : 0.9892
Logloss  : 0.1340

Sample predictions (first 10):
[6.650e-02 4.000e-03 9.848e-01 9.843e-01 2.000e-04 4.890e-02 8.000e-04
 9.824e-01 9.808e-01 9.997e-01]

[Info] Derived categorical indices: [0, 1, 2]
[Info] Derived continuous indices : [3, 4, 5, 6, 7, 8, 9, 10, 11, 12] ... (total: 10 )
[Info] Cardinalities